In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from fmu.sumo.explorer import AggregatedTable, Explorer

%matplotlib inline

# These examples use Seaborn for plotting. Seaborn does not automatically install with
# fmu-sumo. If your environment does not have Seaborn installed, uncomment line below.

# !pip install seaborn
import seaborn as sns

### Initialize Explorer

In [ ]:
sumo = Explorer("dev")

### Get Case

In [ ]:
# Get case by name (name is not guaranteed to be unique)
case = sumo.cases.filter(uuid="5e6bd69f-eaa2-49b7-b323-62a84d533051")[0]
case.name

### Finding info about tables connected to case


In sumo a big group of data are stored as tables. Any data that does not have spacial references will have to be in this format.<br>
This means that this is the solution for datatypes such as inplace volumes, and summary data from reservoir simulators. <br>
Even data that has spacial reference might be stored as a table. So to be general one can say that as long as the data is in a table like format,<br> like in an excel or oocalc spreadsheet/pandas dataframe/arrow table or something similar you can upload it to sumo as a table. <br>

When it comes to metadata there is a very logical triplet for surfaces/polygons/points, these are name/tagname/content <br>
This is very influenced by how these are stored in rms, where they refer to name of horizon/representation/and content type.<br>
For example for a surface the name could be BCU (Base Cretaceous unconformity), the representation could be time or depth, and the content could be a <br>2D grid or point sets. 

For tables this is a bit more unclear, so one will for example find tables that have an empty tag name. <br>
Whatever there is of a convention here for now can be described as:<br>

**inplace volumes coming out of rms**<br>
name: name of grid in rms <br>
tagname: vol <br>
content: volumes<br>

**tables from eclipse** (results extracted with package ecl2df)<br>
name: name of datafile (but no -<realization number>)<br>
tagname: datatype as according to ecl2df <br>
content: summary data will get timeseries, but for now the rest will get property <br>
There is a suggestion (issue in the fmu-dataio repo) that basically the content will be same as the tagname, meaning rft will be content rft<br>
pvt will be pvt etc. But here there are inconsistencies, e.g. relperm data is called satfunc in ecl2df, and it seems more logical to use relperm, <br>
or relativepermeability for this type. Here it would be good with input from the domain experts.
 for name the convention adopted is that for data coming out of <br>

**Any other table**<br>
Here there is really anarchy, so it is up to the end user to define this themselves when exporting with fmu-dataio.


In [ ]:
tables = case.tables
print(f"Table names: {tables.names}")
print(f"Table tags: {tables.tagnames}")

#### Fetching one table

Most tables are stored during the running of an fmu run. So you will in that case have one version per realization (i.e per file path realization-n/iter-m) <br>
This means that it will be a slow process to get to results from one ensemble. <br>
Below is an example of how to fetch one table. General syntax: <br>
``table_collection = case.tables.filter(realization=<realization number>, ensemble=<ensemble name>,``<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;``name=<table name>, tagname=<name of tag>)``<br>
Any of the input arguements are optional, so you could end up with more than one resulting table, and in theory even if you have <br>
used all arguements you could end up with several (long story, please ask). But you can check how many tables you have with using <br>
``len(table_collection)``<br>
To get to one table you can then access them using indexing as you would with a list in python: <br>
``table = table_collection[0]``<br>
If you want to access the actual object, you can access this with the methods to_pandas or to_arrow, and you also have access to the <br>
metadata via indexing as you would do a dictionary. The entire metadata can be access with using the attribute _metadata



In [ ]:
# Filter using the key
one_table = tables.filter(
    realization=0, ensemble="iter-0", name="DROGON", tagname="compdat"
)[0]

# Give back the name and tag
print(f"Found table {one_table.name}-{one_table.tagname}")

# fetching the actual table as a Pandas dataframe
print(one_table.name)
print(one_table.to_pandas().head())

# If you know the metadata fields you can access the data directly on the object with square brackets
print(one_table["data"])

Since it is slow to aggregate the tables yourself, and that the tables then would become very large Sumo comes with a service that aggregates the tables, and then splits them up by column. We refer to these objects as aggregated tables even though they are both aggregated and split up. These come in different types, but the most general is the collection, where you get all realizations stacked on top of each other,<br> but you also have access to statistical aggregations such as mean, min,max, std, p10, and p90 <br>

Below is described how one uses these.

### Working with aggregated tables
There are two ways of getting to aggregated tables
1. The filtering way
2. Through the AggregatedTable class (Which is using filtering under the hood)


### The filtering way

General syntax:<br>
``selection = case.tables.filter(name=<NAME OF TABLE>, tagname=<NAME OF TAG>, column=<COLUMN NAME>, aggregation=<aggregation type>)``

All these are optional as explained above, but you have to have the aggregation argument to get to an aggregated object. <br>
And if you leave out some of them you will end up with a collection of them, not just one

##### Filtering to get to all aggregated tables with name of data file, for the drogon case it is called DROGON

In [ ]:
sim_tables = tables.filter(
    name="DROGON", ensemble="iter-0", aggregation="collection"
)
sim_tables.tagnames

## RFT

In [ ]:
rft_tables = sim_tables.filter(tagname="rft")

The rft table object now contains all the aggregated tables relating to the rft tag. <br>
The different object names you can access via the columns attribute, sort of similar to a pandas dataframe

In [ ]:
rft_tables.columns

To get to one spesific table you use the filtering option column=, <br> and fetch the element just like in a list<br>
Which gives access to one object, but this can be accessed both as pandas dataframe and pyarrow table

In [ ]:
pressure = rft_tables.filter(column="PRESSURE")[0]
frame = pressure.to_pandas()
print(
    f"The following columns are in the pressure object {frame.columns.to_list()}"
)

#### After this it is easy to make a plot

In [ ]:
names = frame.WELL.unique()
dates = frame.DATE.unique()
fig, plots = plt.subplots(len(dates), len(names))

for i, date in enumerate(dates):
    for j, well in enumerate(names):
        data = frame.loc[
            (date == frame.DATE) & (well == frame.WELL)
        ].sort_values(by="DEPTH")
        ax = plots[i, j]
        if data.empty:
            ax = plots[i, j]  # get current axes
            ax.get_xaxis().set_visible(False)  # hide x-axis
            ax.get_yaxis().set_visible(False)  # hide y-axis
            ax.axis("off")
        else:
            data[["DEPTH", "PRESSURE"]].plot(ax=ax, x="PRESSURE", y="DEPTH")
            ax.get_legend().remove()
            if i == 0:
                ax.set_title(well)

            ax.invert_yaxis()


plt.show()

### For even more user friendly access to summary data


##### Use the class Aggregated Table 

``COLLECTION = AggregatedTable(case, <NAME OF DATAFILE>, <DATATYPE>)``

Here ``<DATATYPE>`` can be of collection, index, mean, min, max, p10 or p90

This class gives you the aggregated tables that share name and tagname for one ensemble as one object, <br> 
so you don't need to know that what you are dealing with is a collection of objects


### Grid

In [ ]:
GRID = AggregatedTable(case, "DROGON", "grid", "iter-0")
GRID["PORO"].to_pandas().plot(kind="hist", y="PORO")
GRID["PERMX"].to_pandas().head()

### Equil

In [ ]:
EQUIL = AggregatedTable(case, "DROGON", "equil", "iter-0")
CONTACT_TYPE = "OWC"
sns.boxplot(
    pd.pivot_table(
        EQUIL[CONTACT_TYPE].to_pandas(),
        index="REAL",
        columns="EQLNUM",
        values=CONTACT_TYPE,
    ).values
)
plt.show()

### RELPERM

In [ ]:
RELPERM = AggregatedTable(case, "DROGON", "satfunc", "iter-0")

KRW = (
    pd.concat((RELPERM["KRW"].to_pandas(), RELPERM["SW"].to_pandas()), axis=1)
    .T.drop_duplicates()
    .T
)
print(KRW.head())

#### A plot

In [ ]:
ax = sns.lineplot(
    KRW.loc[(KRW.KEYWORD == "SWOF")],
    x="SW",
    y="KRW",
    hue="SATNUM",
    style="REAL",
)
ax.legend(loc="right", ncols=6, bbox_to_anchor=(2.1, 0.5))
plt.show()

### Summary

In [ ]:
summary = AggregatedTable(case, "DROGON", "summary", "iter-0")
VECTOR_NAME = "FOIP"
ax = (
    pd.pivot_table(
        summary[VECTOR_NAME].to_pandas(),
        index="DATE",
        columns="REAL",
        values=VECTOR_NAME,
    )
    .dropna(axis=0)
    .plot()
)
ax.get_legend().remove()
ax.set_label(VECTOR_NAME)
plt.show()

### Compdat

In [ ]:
COMPDAT = AggregatedTable(case, "DROGON", "compdat", ensemble="iter-0")
COMPDAT["KH"].to_pandas()

### Wellcompletions

In [ ]:
COMPLETIONS = AggregatedTable(case, "DROGON", "wellcompletiondata", "iter-0")
KH = COMPLETIONS["KH"].to_pandas()
KH.head()

In [ ]:
KH["ZONE_NR"] = KH["ZONE"].replace(
    {
        value: key
        for key, value in dict(enumerate(KH["ZONE"].unique().tolist())).items()
    }
)
MEAN_STD = pd.pivot_table(
    KH,
    index=["ZONE_NR", "ZONE"],
    columns="WELL",
    values="KH",
    aggfunc=["mean", "std"],
)
# KH.head()
MEAN_STD[("mean",)]["A1"]

#### A plot

In [ ]:
sns.scatterplot(KH, x="WELL", y="ZONE", hue="KH")

### FIPREPORTS

In [ ]:
REPORTS = AggregatedTable(case, "DROGON", "fipreports", "iter-0")
print(REPORTS.columns)
REPORT_NAME = "STOIIP_OIL"
STOIIP = REPORTS[REPORT_NAME].to_pandas().dropna(subset=REPORT_NAME, axis=0)
STOIIP.head()

### Faults
Seems to be something wrong with it, will need to have a look


In [ ]:
FAULTS = AggregatedTable(case, "DROGON", "faults", "iter-0")
print(FAULTS.columns)
COMPLETE = pd.concat(
    (FAULTS["I"].to_pandas(), FAULTS["J"].to_pandas(), FAULTS["K"].to_pandas())
)
COMPLETE.head()

### The table index 

Throughout the examples of the aggregated tables you have probably noticed that the aggregated tables comes with additonal columns apart from the <br>
specific one that you have asked for. E.g. fetching an aggregated table for FOPT from summary will include the DATE column, and if you ask from PORO <br> from grid you will get these additional elements: ["GLOBAL_INDEX", "I", "J", "K"]. This is enables you to only download one object and you straight out of the box have all you need for plotting or analysis. These could be revised though, and here we want input from the users.
 E.g. for now satfunc (which is relperm) does not include SW, should that be included? These are the definitions as of now, <br>
 please give feedback on this!!

DEFINITIONS = {<br>
&emsp;    "inplace_volumes": ["ZONE", "REGION", "FACIES", "LICENCE"], <br>
&emsp;    "wellpicks": ["WELL", "HORIZON"], <br>
&emsp;    "summary": ["DATE"], <br>
&emsp;    "equil": ["EQLNUM"], <br>
&emsp;    "compdat": ["WELL", "DATE", "DIR"], <br>
&emsp;    "faults": ["NAME", "FACE"], <br>
&emsp;    "fipreports": ["DATE", "FIPNAME", "REGION"], <br>
&emsp;    "grid": ["GLOBAL_INDEX", "I", "J", "K"], <br>
&emsp;    "pillars": ["PILLAR"], <br>
&emsp;    "pvt": ["PVTNUM", "KEYWORD"], <br>
&emsp;    "rft": ["WELL", "DATE", "DEPTH"], <br>
&emsp;    "satfunc": ["SATNUM", "KEYWORD"], <br>
&emsp;    "wellcompletiondata": ["WELL", "DATE", "ZONE"], <br>
}